In [2]:
!pip install mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 23.6 MB/s eta 0:00:00


In [3]:
# Import necessary libraries
import os
import numpy as np
import mne
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload

# Authenticate and create the Google Drive API service
auth.authenticate_user()
drive_service = build('drive', 'v3')
eeg_channels = ['Cz', 'Fz', 'Fp1', 'F7', 'F3', 'FC1', 'C3', 'FC5', 'FT9', 'T7', 'CP5', 'CP1', 'P3', 'P7', 'PO9', 'O1', 'Pz', 'Oz', 'O2', 'PO10', 'P8', 'P4', 'CP2', 'CP6', 'T8', 'FT10', 'FC6', 'C4', 'FC2', 'F4', 'F8', 'Fp2']

def list_files_in_folder(service, folder_id):
    try:
        # List all files in the specified folder
        results = service.files().list(
            q=f"'{folder_id}' in parents",
            fields="nextPageToken, files(id, name, mimeType)"
        ).execute()

        items = results.get('files', [])

        if not items:
            print('No files found in the folder.')
        else:
            print('Files and folders in the folder:')
            for item in items:
                print(f'{item["name"]} ({item["id"]}) - {item["mimeType"]}')
    except HttpError as error:
        print(f'An error occurred: {error}')
        items = None

    return items

def process_csv(file_id, service):
    try:
        request = service.files().get_media(fileId=file_id)
        csv_path = f'/content/{file_id}.csv'
        with open(csv_path, 'wb') as file:
            downloader = MediaIoBaseDownload(file, request)
            done = False
            while not done:
                status, done = downloader.next_chunk()
                print(f'Download {int(status.progress() * 100)}%.')

        with open(csv_path, 'r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                try:
                    print(row['stim'], row['agree.response'], row['emotion'])
                except KeyError as e:
                    print(f"Column not found: {e}")
    except HttpError as error:
        print(f'An error occurred: {error}')

def process_edf(file_id, service):
    try:
        request = service.files().get_media(fileId=file_id)
        edf_path = f'/content/{file_id}.edf'
        with open(edf_path, 'wb') as file:
            downloader = MediaIoBaseDownload(file, request)
            done = False
            while not done:
                status, done = downloader.next_chunk()
                print(f'Download {int(status.progress() * 100)}%.')

        # Load .edf file using MNE
        raw = mne.io.read_raw_edf(edf_path)
        print('EDF PATH:\n\n\n\n\n',edf_path)
        raw.pick_types(eeg=True)
        montage = mne.channels.make_standard_montage('standard_1020')
        raw.set_montage(montage, on_missing='ignore')
        # Plot PSD
        raw.plot_psd(fmax=50,picks=eeg_channels)
    except HttpError as error:
        print(f'An error occurred: {error}')

def find_and_process_files(service, folder_id):
    items = list_files_in_folder(service, folder_id)
    for item in items:
        if item['mimeType'] == 'application/vnd.google-apps.folder':
            find_and_process_files(service, item['id'])
        elif item['mimeType'] == 'text/csv':
            print(f'Processing CSV file: {item["name"]} ({item["id"]})')
            process_csv(item['id'], service)
        elif item['mimeType'] == 'application/octet-stream':
            print(f'Processing EDF file: {item["name"]} ({item["id"]})')
            process_edf(item['id'], service)

# Specify the ID of the shared folder
folder_id = '1QLBrAlsjDKowTr3-__iId9-v9gbAI5PZ'

# Find and process all files in the shared folder
find_and_process_files(drive_service, folder_id)
eeg_channels = ['Cz', 'Fz', 'Fp1', 'F7', 'F3', 'FC1', 'C3', 'FC5', 'FT9', 'T7', 'CP5', 'CP1', 'P3', 'P7', 'PO9', 'O1', 'Pz', 'Oz', 'O2', 'PO10', 'P8', 'P4', 'CP2', 'CP6', 'T8', 'FT10', 'FC6', 'C4', 'FC2', 'F4', 'F8', 'Fp2']
# Read your EDF file
raw = mne.io.read_raw_edf(r'/content/1IomsFvwQPZWOnbTnEBWkxt6zq2ZfDqNt.edf', preload=True, verbose=True, infer_types=True)#, infer_types=True, include=eeg_channels
ecg_epochs = mne.preprocessing.create_ecg_epochs(raw)
ecg_epochs.plot_image(combine="mean")

MessageError: Error: credential propagation was unsuccessful